# Basic Example: Fairlex Calibration

This example illustrates how to construct a membership matrix and target
totals, call the weight-fair leximin calibration, and inspect the resulting
weights and diagnostics.

In [ ]:
import numpy as np
from fairlex import evaluate_solution, leximin_weight_fair

## Setting up the problem

Suppose we survey five people and want to calibrate on sex and age.
Each margin is represented by two rows: the indicator for the
first category and the second category. We also include a total row.

In [ ]:
# Define the membership matrix A
A = np.array(
    [
        # sex: female
        [1, 0, 1, 0, 1],
        # sex: male
        [0, 1, 0, 1, 0],
        # age: young (<=40)
        [1, 1, 0, 0, 1],
        # age: old (>40)
        [0, 0, 1, 1, 0],
        # total
        [1, 1, 1, 1, 1],
    ],
    dtype=float,
)

print("Membership matrix A:")
print(A)

In [ ]:
# Base weights (e.g. equal weights in a simple random sample)
w0 = np.ones(5)
print("Base weights:", w0)

# Target totals for the population (feasible with max weight 2.0 per person)
# Note: with 5 people and max weight 2.0, total achievable is 10
target = np.array([6, 4, 6, 4, 10], dtype=float)
print("Target totals:", target)

## Performing calibration

We use the weight-fair leximin calibration with modest weight bounds.

In [ ]:
# Perform calibration with modest bounds
result = leximin_weight_fair(A, target, w0, min_ratio=0.5, max_ratio=2.0)
print("Calibrated weights:", result.w)
print("Optimization status:", result.status)
print("Solver message:", result.message)

## Evaluating the solution

Let's examine the quality of our calibrated weights using various diagnostics.

In [ ]:
# Evaluate solution quality
metrics = evaluate_solution(A, target, result.w, base_weights=w0)

print("Solution Diagnostics:")
print("====================")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

## Understanding the results

The diagnostics show us:
- **Residual metrics**: How well we achieved our target totals
- **Weight distribution**: Statistical properties of the final weights
- **Relative deviations**: How much the weights changed from their base values
- **ESS (Effective Sample Size)**: Measure of variance inflation
- **Design Effect**: Ratio of actual to nominal sample size